In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
mnist = tf.keras.datasets.mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
index_train = np.where((train_labels == 2) | (train_labels == 6))
index_test = np.where((test_labels == 2) | (test_labels == 6) )
train_images_26 = train_images[index_train]
train_images_26 = train_images_26.reshape((len(train_images_26),train_images_26[1].size))
train_labels_26 = train_labels[index_train].astype('int')
test_images_26 = test_images[index_test]
test_images_26 = test_images_26.reshape((len(test_images_26),test_images_26[1].size))
test_labels_26 = test_labels[index_test].astype('int')
train_labels_26[ np.where(train_labels_26 == 2 )] = -1
train_labels_26[ np.where(train_labels_26 == 6 )] = 1
test_labels_26[ np.where(test_labels_26 == 2 )] = -1
test_labels_26[ np.where(test_labels_26 == 6 )] = 1
(train_images_26)= train_images_26[range(1000)].astype(np.float64)
(train_labels_26)= train_labels_26[range(1000)].astype(np.float64)

train_images_26 = train_images_26/255
test_images_26 = test_images_26/255



In [6]:
import cvxpy as cp
from sklearn.utils import shuffle

In [7]:
def kernel_svm(sigma,train_images_26_norm,train_labels_26_norm,test_images_26_norm,test_labels_26_norm):
  a_kernel = cp.Variable((train_labels_26_norm.shape[0]))


  def dot_mat_vector(x):
    mat = np.zeros((len(x), len(x)))
    for i in range(len(x)):
      for j in range(len(x)):
        mat[i][j]= product(x[i],x[j])
    return mat

  def product(c,d):
    
    #sigma = 1
    num = -1*np.sum(np.square(c-d))
    denom = 2*sigma**2
    return np.exp((num/denom))
    

  def dot_mat(x):
    mat = np.zeros((len(x), len(x)))
    for i in range(len(x)):
      for j in range(len(x)):
        mat[i][j]= x[i]*x[j]
    return mat

  mat_x = np.zeros((len(train_images_26_norm), len(train_images_26_norm))).astype(np.float64)
  mat_x = dot_mat_vector(train_images_26_norm)

  mat_y = np.zeros((len(train_labels_26_norm), len(train_labels_26_norm)))
  mat_y = dot_mat(train_labels_26_norm)

  h = np.zeros((train_images_26_norm.shape[0], train_images_26_norm.shape[0])).astype(np.float64)
  h = np.multiply(mat_x,mat_y)


  obj = cp.Maximize(cp.sum(a_kernel)-0.5*cp.quad_form(a_kernel,h))
  c2 = [a_kernel@train_labels_26_norm.T == 0]
  a_const = [a_kernel[i] >=0 for i in range(train_labels_26_norm.shape[0])]

  constraints = c2+a_const
  prob=cp.Problem(obj,constraints)
  prob.solve()
  print("status:", prob.status)
  #print("optimal value", prob.value)


  def summation(x):
    w_sum = 0
    
    for i in range(len(train_labels_26_norm)):
      w_sum += train_labels_26_norm[i]*a_kernel.value[i]*product(train_images_26_norm[i],x)
    return w_sum
  max = -np.inf
  min = np.inf
  for i in range(len(train_images_26_norm)):
    if ( train_labels_26_norm[i] == -1):
      if summation(train_images_26_norm[i]) > max :
        max = summation(train_images_26_norm[i])
    else:
      if summation(train_images_26_norm[i]) < min :
        min = summation(train_images_26_norm[i])

  b_kernel = (-1*(max + min))/2

  # incorr = 0
  # for j in range(len(train_images_26_norm)): 
  #       wxtest = summation(train_images_26_norm[j]) + b_kernel
  #       if wxtest*train_labels_26_norm[j]<=0:
  #         incorr += 1
  # kernel_training_error = ((incorr)/len(train_images_26_norm))
  # print("training error :",kernel_training_error)

  incorr = 0

  for j in range(len(test_images_26_norm)): 
        wxtest = summation(test_images_26_norm[j]) + b_kernel
        if wxtest*test_labels_26_norm[j]<=0:
          incorr += 1
  kernel_testing_error = ((incorr)/len(test_images_26_norm))
  print("testing error :",kernel_testing_error)
  return kernel_testing_error

In [8]:
sig = [0.2, 0.5, 1, 3, 4, 5, 10]
#kernel_training_error=[]
kernel_testing_error=[]
train_images_shuffle,train_labels_shuffle= shuffle(train_images_26,train_labels_26)
for i in sig:
  print(f"Testing with sigma = {i}")
  error = []
  for k in range(10):
    index = [i for i in range((k*100),((k+1)*100))]

    train_images_26_norm = train_images_shuffle[[j for j in range(len(train_images_shuffle)) if j not in index]]
    train_labels_26_norm = train_labels_shuffle[[j for j in range(len(train_labels_shuffle)) if j not in index]]

    test_images_26_norm = train_images_shuffle[[j for j in range(len(train_images_shuffle)) if j in index]]
    test_labels_26_norm = train_labels_shuffle[[j for j in range(len(train_labels_shuffle)) if j in index]]
    
    error.append(kernel_svm(i,train_images_26_norm,train_labels_26_norm,test_images_26_norm,test_labels_26_norm))
  
  kernel_testing_error.append(np.mean(error))
    

print(kernel_testing_error)

Testing with sigma = 0.2
status: optimal
testing error : 0.54
status: optimal
testing error : 0.49
status: optimal
testing error : 0.45
status: optimal
testing error : 0.53
status: optimal
testing error : 0.57
status: optimal
testing error : 0.48
status: optimal
testing error : 0.48
status: optimal
testing error : 0.46
status: optimal
testing error : 0.45
status: optimal
testing error : 0.46
Testing with sigma = 0.5
status: optimal
testing error : 0.54
status: optimal
testing error : 0.49
status: optimal
testing error : 0.45
status: optimal
testing error : 0.53
status: optimal
testing error : 0.57
status: optimal
testing error : 0.48
status: optimal
testing error : 0.48
status: optimal
testing error : 0.46
status: optimal
testing error : 0.45
status: optimal
testing error : 0.46
Testing with sigma = 1
status: optimal
testing error : 0.54
status: optimal
testing error : 0.49
status: optimal
testing error : 0.45
status: optimal
testing error : 0.53
status: optimal
testing error : 0.57
st

In [9]:
from tabulate import tabulate 
table = [["Test","Training Error","Testing Error"],\
         ["K-SVM sig = 0.2 ",kernel_testing_error[0]],\
         ["K-SVM sig = 0.5 ",kernel_testing_error[1]],\
         ["K-SVM sig = 1.0 ",kernel_testing_error[2]],\
         ["K-SVM sig = 3.0 ",kernel_testing_error[3]],\
         ["K-SVM sig = 4.0 ",kernel_testing_error[4]],\
         ["K-SVM sig = 5.0 ",kernel_testing_error[5]],\
         ["K-SVM sig = 10.0 ",kernel_testing_error[6]]
         ]
#print(tabulate(table))
print(tabulate(table,headers="firstrow",tablefmt="fancy_grid"))
min_error_sigma = sig[kernel_testing_error.index(min(kernel_testing_error))]
print(f"Best model is with sigma {min_error_sigma}")
print(f"Training on entire data and testing error on validation set is {kernel_svm(min_error_sigma,train_images_26,train_labels_26,test_images_26,test_labels_26)}")

╒══════════════════╤══════════════════╕
│ Test             │   Training Error │
╞══════════════════╪══════════════════╡
│ K-SVM sig = 0.2  │            0.491 │
├──────────────────┼──────────────────┤
│ K-SVM sig = 0.5  │            0.491 │
├──────────────────┼──────────────────┤
│ K-SVM sig = 1.0  │            0.491 │
├──────────────────┼──────────────────┤
│ K-SVM sig = 3.0  │            0.015 │
├──────────────────┼──────────────────┤
│ K-SVM sig = 4.0  │            0.005 │
├──────────────────┼──────────────────┤
│ K-SVM sig = 5.0  │            0.004 │
├──────────────────┼──────────────────┤
│ K-SVM sig = 10.0 │            0.014 │
╘══════════════════╧══════════════════╛
Best model is with sigma 5
status: optimal
testing error : 0.007537688442211055
Training on entire data and testing error on validation set is 0.007537688442211055


K-fold cross validation results also say that sigma = 5 is the best gamma value for kernal SVM similar to simple cross validation results.